In [1]:
pip install featuretools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.2/619.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.4/754.4 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: holidays
    Found existing installation: holidays 0.36
    Uninstalling holidays-0.36:
      Successfully uninstalled holidays-0.36


In [2]:
import numpy as np
import featuretools as ft
import pandas as pd

In [3]:
from featuretools.primitives.standard.transform.datetime.season import date

customers = pd.read_csv('/content/customers.csv',
                        dtype ={'CustomerId': int, 'Name' : str, 'Email' :str , 'SignupDate':str},
                        parse_dates=['SignupDate'])
products = pd.read_csv('/content/products.csv',
                       dtype = {'ProductID': int, 'Name' : str, 'Category' :str , 'Price':float})
orders = pd.read_csv('/content/orders.csv',
                     dtype = {'OrderID': int, 'CustomerID' : int, 'OrderDate' :str , 'ShipDate':str},
                     parse_dates=['OrderDate','ShipDate'] )
orderDetails = pd.read_csv('/content/orderDetails.csv',
                           dtype = {'OrderID': int, 'ProductID' : int, 'Quantity' :int , 'Discount':float})

#remove any nan column
customers = customers.dropna()
products = products.dropna()
orders = orders.dropna()
orderDetails = orderDetails.dropna()

In [4]:
#create entity and entityset
es = ft.EntitySet(id = 'orderDetails')

#use existing index
es = es.add_dataframe(dataframe_name= 'products', dataframe = products, index = 'ProductID')
es = es.add_dataframe(dataframe_name= 'customers', dataframe = customers, index = 'CustomerID')
es = es.add_dataframe(dataframe_name= 'orders', dataframe = orders, index = 'OrderID')
#create new unique index
es = es.add_dataframe(dataframe_name= 'orderDetails', dataframe = orderDetails,
    make_index = True, index = 'orderDetails_index')

es

Entityset: orderDetails
  DataFrames:
    products [Rows: 10, Columns: 4]
    customers [Rows: 15, Columns: 4]
    orders [Rows: 20, Columns: 4]
    orderDetails [Rows: 20, Columns: 5]
  Relationships:
    No relationships

In [5]:
#create relationship
es = es.add_relationship('customers', 'CustomerID', 'orders', 'CustomerID')
es = es.add_relationship('orders', 'OrderID', 'orderDetails', 'OrderID')
es = es.add_relationship('products','ProductID', 'orderDetails', 'ProductID')

es

Entityset: orderDetails
  DataFrames:
    products [Rows: 10, Columns: 4]
    customers [Rows: 15, Columns: 4]
    orders [Rows: 20, Columns: 4]
    orderDetails [Rows: 20, Columns: 5]
  Relationships:
    orders.CustomerID -> customers.CustomerID
    orderDetails.OrderID -> orders.OrderID
    orderDetails.ProductID -> products.ProductID

In [6]:
#Determine primitive
agg_primitives =  ["sum","count", "percent_true", "mode"]
trans_primitives =  ["day", "year", "month", "weekday", "haversine","num_words", "num_characters",
                     "subtract_numeric","add_numeric","multiply_numeric"]

#DFS with specified primitives
feature_matrix, feature_defs = ft.dfs(entityset = es,
    target_dataframe_name = 'orderDetails',
    trans_primitives = trans_primitives,
    agg_primitives=agg_primitives,
    max_depth = 4, n_jobs = -1, verbose = 1)

2023-11-13 04:10:19,395 featuretools - WARNING    Attempting to add feature <Feature: customers.COUNT(orderDetails) + SUM(orderDetails.Discount)> which is already present. This is likely a bug.
2023-11-13 04:10:19,396 featuretools - WARNING    Attempting to add feature <Feature: customers.COUNT(orderDetails) + SUM(orderDetails.Quantity)> which is already present. This is likely a bug.
2023-11-13 04:10:19,398 featuretools - WARNING    Attempting to add feature <Feature: customers.COUNT(orders) + SUM(orderDetails.Discount)> which is already present. This is likely a bug.
2023-11-13 04:10:19,399 featuretools - WARNING    Attempting to add feature <Feature: customers.COUNT(orders) + SUM(orderDetails.Quantity)> which is already present. This is likely a bug.
2023-11-13 04:10:19,400 featuretools - WARNING    Attempting to add feature <Feature: customers.SUM(orderDetails.Discount) + SUM(orderDetails.Quantity)> which is already present. This is likely a bug.
2023-11-13 04:10:19,401 featuretool

/usr/local/lib/python3.10/dist-packages/featuretools/synthesis/dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  trans_primitives: ['haversine', 'num_characters', 'num_words']
  agg_primitives: ['percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Built 543 features
Elapsed: 00:00 | Progress:   0%|          

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:46707
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42987'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46367'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:38743', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:38743
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48366
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:43745', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:43745
INFO:distributed.core:Sta

EntitySet scattered to 2 workers in 3 seconds
Elapsed: 00:02 | Progress:  95%|█████████▌

INFO:distributed.scheduler:Remove client Client-90d145a3-81da-11ee-80f1-0242ac1c000c
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:59184; closing.
INFO:distributed.scheduler:Remove client Client-90d145a3-81da-11ee-80f1-0242ac1c000c
INFO:distributed.scheduler:Close client connection: Client-90d145a3-81da-11ee-80f1-0242ac1c000c
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:42987'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.nanny:Closing Nanny at 'tcp://127.0.0.1:46367'. Reason: nanny-close
INFO:distributed.nanny:Nanny asking worker to close. Reason: nanny-close
INFO:distributed.core:Received 'close-stream' from tcp://127.0.0.1:48366; closing.
INFO:distributed.scheduler:Remove worker <WorkerState 'tcp://127.0.0.1:38743', name: 0, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1699848628.638726')
INFO:distributed.batched:Batched Comm Closed <TCP (closed) Schedu

Elapsed: 00:05 | Progress: 100%|██████████


In [7]:
feature_matrix

,OrderID,ProductID,Quantity,Discount,Discount + Quantity,Discount * Quantity,Discount - Quantity,orders.CustomerID,products.Price,orders.COUNT(orderDetails),...,orders.customers.SUM(orderDetails.Quantity) - products.COUNT(orderDetails),orders.customers.SUM(orderDetails.Quantity) - products.Price,orders.customers.SUM(orderDetails.Quantity) - products.SUM(orderDetails.Discount * Quantity),orders.customers.SUM(orderDetails.Quantity) - products.SUM(orderDetails.Discount + Quantity),orders.customers.SUM(orderDetails.Quantity) - products.SUM(orderDetails.Discount - Quantity),orders.customers.SUM(orderDetails.Quantity) - products.SUM(orderDetails.Discount),orders.customers.SUM(orderDetails.Quantity) - products.SUM(orderDetails.Quantity),products.SUM(orderDetails.Discount * Quantity) - Quantity,products.SUM(orderDetails.Discount + Quantity) - Quantity,products.SUM(orderDetails.Discount - Quantity) - Quantity
orderDetails_index,,,,,,,,,,,,,,,,,,,,,
0,301,201,1,0.00,1.00,0.00,-1.00,101,500.0,1,...,1.0,-497.0,3.0,1.0,5.0,3.0,1.0,-1.0,1.0,-3.0
1,302,202,2,0.10,2.10,0.20,-1.90,102,300.0,1,...,1.0,-297.0,2.6,-1.2,6.8,2.8,-1.0,-1.6,2.2,-5.8
2,303,203,1,0.00,1.00,0.00,-1.00,103,700.0,1,...,0.0,-698.0,2.0,0.0,4.0,2.0,0.0,-1.0,1.0,-3.0
3,304,204,3,0.20,3.20,0.60,-2.80,104,20.0,1,...,3.0,-15.0,3.8,-1.4,10.6,4.6,-1.0,-1.8,3.4,-8.6
4,305,205,1,0.00,1.00,0.00,-1.00,105,50.0,1,...,0.0,-48.0,2.0,0.0,4.0,2.0,0.0,-1.0,1.0,-3.0
5,306,206,2,0.00,2.00,0.00,-2.00,106,80.0,1,...,0.0,-78.0,2.0,-2.0,6.0,2.0,-2.0,-2.0,2.0,-6.0
6,307,207,1,0.15,1.15,0.15,-0.85,107,1000.0,1,...,-1.0,-999.0,0.7,-1.3,2.7,0.7,-1.0,-0.7,1.3,-2.7
7,308,208,1,0.00,1.00,0.00,-1.00,108,800.0,1,...,-1.0,-799.0,1.0,-1.0,3.0,1.0,-1.0,-1.0,1.0,-3.0
8,309,209,2,0.10,2.10,0.20,-1.90,109,150.0,1,...,0.0,-148.0,1.6,-2.2,5.8,1.8,-2.0,-1.6,2.2,-5.8


In [8]:
feature_defs

[<Feature: OrderID>,
 <Feature: ProductID>,
 <Feature: Quantity>,
 <Feature: Discount>,
 <Feature: Discount + Quantity>,
 <Feature: Discount * Quantity>,
 <Feature: Discount - Quantity>,
 <Feature: orders.CustomerID>,
 <Feature: products.Price>,
 <Feature: orders.COUNT(orderDetails)>,
 <Feature: orders.SUM(orderDetails.Discount)>,
 <Feature: orders.SUM(orderDetails.Quantity)>,
 <Feature: orders.DAY(OrderDate)>,
 <Feature: orders.DAY(ShipDate)>,
 <Feature: orders.MONTH(OrderDate)>,
 <Feature: orders.MONTH(ShipDate)>,
 <Feature: orders.WEEKDAY(OrderDate)>,
 <Feature: orders.WEEKDAY(ShipDate)>,
 <Feature: orders.YEAR(OrderDate)>,
 <Feature: orders.YEAR(ShipDate)>,
 <Feature: products.COUNT(orderDetails)>,
 <Feature: products.SUM(orderDetails.Discount)>,
 <Feature: products.SUM(orderDetails.Quantity)>,
 <Feature: Discount + products.Price>,
 <Feature: products.Price + Quantity>,
 <Feature: Discount * products.Price>,
 <Feature: products.Price * Quantity>,
 <Feature: Discount - products.Pri

In [9]:
feature_matrix.to_csv('/content/result.csv', index=False)